Please head over to this website to view the results: https://walterra.github.io/covid-19/

*I really appreciate if you're interested in the code, however, proceed with caution: All of this is a bunch of spaghetti code that gets the job done. There's lot of room to improve things and make the Python code more readable and simple. (It wouldn't change a lot about the generated VEGA specs though, they are fine, thanks Altair!*

In [1]:
import altair as alt
import json
import pandas as pd

In [2]:
df = pd.read_csv("../data/corin_data.csv")
# Use this line if only morning data is available and not the full day (removes last row)
# df = df[:-1]
df['dataTime'] = pd.to_datetime(df['dataTime'], format='%d.%m.%Y %H:%M:%S')
df = df[['dataTime', 'total_confirmed', 'total_death', 'total_recovered', 'total_tests']]
df = df.fillna(method='ffill')

# Ignore the 9:00 numbers
df = df[df['dataTime'].dt.hour > 12]

df['total_active'] = df['total_confirmed'] - df['total_recovered'] - df['total_death']
pd.set_option('display.max_rows', None)
df.head(10)

,dataTime,total_confirmed,total_death,total_recovered,total_tests,total_active
0,2020-02-25 14:30:00,2,0,0.0,218,2.0
2,2020-02-27 17:15:00,3,0,0.0,447,3.0
3,2020-02-28 16:30:00,6,0,0.0,763,6.0
6,2020-03-02 18:00:00,18,0,0.0,2683,18.0
8,2020-03-03 16:00:00,24,0,0.0,2683,24.0
9,2020-03-04 18:00:00,29,0,0.0,3138,29.0
10,2020-03-05 15:00:00,41,0,0.0,3711,41.0
11,2020-03-06 15:00:00,55,0,0.0,4000,55.0
12,2020-03-07 15:00:00,79,0,0.0,4308,79.0
14,2020-03-09 15:00:00,131,0,2.0,4734,129.0


In [3]:
df_by_day = df

In [4]:
def c_line(att='total_confirmed',col='blue',scaleType='linear',label='value',labelOffset=0):
    the_base = alt.Chart(df, width=300, height=200).transform_filter(
        alt.datum[att] > 0  
    ).transform_timeunit(
        date='yearmonthdate(dataTime)'
    ).transform_aggregate(
        max_att='max('+att+')',
        groupby=['date']
    )
    
    the_chart = the_base.mark_line(
        color=col,
        size=2,
        strokeJoin='round'
    ).encode(
        alt.X('date:T', title=''),
        alt.Y('max_att:Q', scale=alt.Scale(type=scaleType, base=10), title='', axis=alt.Axis(minExtent=50))
    )
    
    the_tooltip = the_base.mark_circle(
        color=col,
        size=200,
        opacity=0
    ).encode(
        alt.X('date:T', title=''),
        alt.Y('max_att:Q', scale=alt.Scale(type=scaleType, base=10), title='', axis=alt.Axis(minExtent=50)),
        tooltip=[alt.Tooltip('date:T', title='Datum'), alt.Tooltip('max_att:Q', title=label)]
    )
    
    the_highlight = the_base.transform_window(
        sort=[alt.SortField("date", order="descending")], 
        rank="rank(date)"
    ).transform_filter(
        alt.datum.rank == 1
    )
    
    the_text = the_highlight.mark_text(
        align='left',
        color=col,
        dx=5,
        dy=labelOffset,
        width=500
    ).encode(
        alt.X('date:T'),
        alt.Y('max_att:Q', scale=alt.Scale(type=scaleType, base=10), title=''),
        text=alt.Text("max_att:Q")
    )
    
    the_mark = the_highlight.mark_circle(
        color=col,
        opacity=1
    ).encode(
        alt.X('date:T'),
        alt.Y('max_att:Q', scale=alt.Scale(type=scaleType, base=10), title=''),
    )
    
    return the_chart + the_text + the_mark + the_tooltip

confirmed_cases = c_line('total_confirmed', '#1f77b4', 'linear', 'Bestätigte Fälle')
deaths = c_line('total_death', '#d62728', 'linear', 'Todesfälle', -5)
recovered = c_line('total_recovered', '#ff7f0e', 'linear', 'Genesene', 5)
active = c_line('total_active', '#17becf', 'linear', 'Aktuell Infizierte', 5)

confirmed_cases_log = c_line('total_confirmed', '#1f77b4', 'log', 'Bestätigte Fälle', -5)
deaths_log = c_line('total_death', '#d62728', 'log', 'Todesfälle', -5)
recovered_log = c_line('total_recovered', '#ff7f0e', 'log', 'Genesene', 5)
active_log = c_line('total_active', '#17becf', 'log', 'Aktuell Infizierte', 5)

linear_scale = alt.layer(active, confirmed_cases, deaths, recovered).properties(title='Linear Scale')
log_scale = alt.layer(active_log, confirmed_cases_log, deaths_log, recovered_log).properties(title='Log Scale')

chart_1_legendDomain = ['Bestätigte Fälle', 'Todesfälle', 'Genesene', 'Aktuell Infizierte']
chart_1_legendColors = ['#1f77b4', '#d62728', '#ff7f0e', '#17becf']
chart_1_legendData = pd.DataFrame({
    'label': chart_1_legendDomain
})

legend1 = alt.Chart(chart_1_legendData).mark_square(size=150).encode(
    y=alt.Y(
        'label:N',
        axis=alt.Axis(domain=False, ticks=False, orient='right'),
        title=None
    ),
    color=alt.Color(
        'label',
        scale=alt.Scale(domain=chart_1_legendDomain,range=chart_1_legendColors),
        legend=None
    )
)

chart_1 = (linear_scale | log_scale | legend1).configure_axis(
    grid=True,
    gridColor="#eee",
    domainColor="#ddd",
    tickColor="#ddd",
    labelColor="gray",
    labelBound=True,
).configure_view(
    strokeWidth=0
).configure_title(
    fontSize=12,
    fontWeight='normal',
    anchor='start',
    color='gray'
)
    

chart_1

alt.HConcatChart(...)

In [24]:
def chart_with_moving_average(valueCalc, title, yAxisFormat, yTextFormat, tooltipLabel):
    c2a_base = alt.Chart(df, width=150, height=150
    ).transform_timeunit(
        date='yearmonthdate(dataTime)'
    ).transform_aggregate(
        max_total_confirmed='max(total_confirmed)',
        max_total_active='max(total_active)',
        groupby=['date']
    ).transform_window(
        rolling_max='max(max_total_confirmed)',
        rolling_min='min(max_total_confirmed)',
        rolling_active='last_value(max_total_active)',
        frame=[-1, 0]
    ).transform_calculate(
        diff=valueCalc
    )

    day = c2a_base.mark_line(
        color='#1f77b4',
        size=2,
        strokeJoin='round'
    ).encode(
        alt.X('date:T', title=''),
        alt.Y('diff:Q', title='', impute=alt.ImputeParams(method='mean')),
        tooltip=['date:T', 'diff:Q']
    )

    c2a_the_tooltip = c2a_base.mark_circle(
        color='#1f77b4',
        size=200,
        opacity=0
    ).encode(
        alt.X('date:T', title=''),
        alt.Y('diff:Q', title=''),
        tooltip=[alt.Tooltip('date:T', title='Datum'), alt.Tooltip(
            'diff:Q', title=tooltipLabel, format=yTextFormat
        )]
    )

    c2a_the_highlight = c2a_base.transform_window(
        sort=[alt.SortField("date", order="descending")], 
        rank="rank(date)"
    ).transform_filter(
        alt.datum.rank == 1
    )

    c2a_the_text = c2a_the_highlight.mark_text(
        align='left',
        color='#1f77b4',
        dx=5,dy=-5,
    ).encode(
        alt.X('date:T'),
        alt.Y('diff:Q', title=''),
        text=alt.Text("diff:Q", format=yTextFormat)
    )

    c2a_the_mark = c2a_the_highlight.mark_circle(
        color='#1f77b4',
        opacity=1
    ).encode(
        alt.X('date:T'),
        alt.Y('diff:Q', title=''),
    )

    ma_base = c2a_base.transform_window(
        rolling_mean='mean(diff)',
        frame=[7, 0]
    )

    ma = ma_base.mark_line(
        color='lightblue',
        size=2,
        strokeJoin='round'
    ).encode(
        alt.X('date:T', title=''),
        alt.Y('rolling_mean:Q', axis=alt.Axis(format=yAxisFormat, title='')),
    )

    c2b_the_tooltip = ma_base.mark_circle(
        color='lightblue',
        size=200,
        opacity=0
    ).encode(
        alt.X('date:T', title=''),
        alt.Y('rolling_mean:Q', title=''),
        tooltip=[alt.Tooltip('date:T', title='Datum'), alt.Tooltip(
            'rolling_mean:Q', title='7d Moving Average', format=yTextFormat
        )]
    )

    c2b_the_highlight = ma_base.transform_window(
        sort=[alt.SortField("date", order="descending")], 
        rank="rank(date)"
    ).transform_filter(
        alt.datum.rank == 1
    )

    c2b_the_text = c2b_the_highlight.mark_text(
        align='left',
        color='lightblue',
        dx=5,dy=5,
    ).encode(
        alt.X('date:T'),
        alt.Y('rolling_mean:Q', title=''),
        text=alt.Text("rolling_mean:Q", format=yTextFormat)
    )

    c2b_the_mark = c2b_the_highlight.mark_circle(
        color='lightblue',
        opacity=1
    ).encode(
        alt.X('date:T'),
        alt.Y('rolling_mean:Q', title='')
    )

    new_cases_by_day = (
        ma + day + c2a_the_text + c2a_the_mark + c2b_the_text + c2b_the_mark + c2b_the_tooltip + c2a_the_tooltip
    ).properties(title=title)
    
    return new_cases_by_day

new_cases_by_day_chart = chart_with_moving_average(
    valueCalc='(datum.rolling_max - datum.rolling_min)',
    title='Neue Fälle pro Tag',
    yAxisFormat='2f',
    yTextFormat='.2f',
    tooltipLabel='Neue Fälle',
)
new_cases_by_day_percent_chart = chart_with_moving_average(
    valueCalc='(datum.rolling_max - datum.rolling_min) / datum.rolling_max',
    title='Steigerung Neue / Bestätigte Fälle',
    yAxisFormat='2%',
    yTextFormat='.2%',
    tooltipLabel='Steigerung'
)

new_cases_by_day_percent_chart2 = chart_with_moving_average(
    valueCalc='(datum.rolling_max - datum.rolling_min) / datum.rolling_active',
    title='Steigerung Neue / Aktive Fälle',
    yAxisFormat='2%',
    yTextFormat='.2%',
    tooltipLabel='Steigerung'
)

legendData = pd.DataFrame({
    'label': ['Tageswert', '7d Moving Average'],
    'color': ['#1f77b4', 'lightblue']
})

legend = alt.Chart(legendData).mark_square(size=150).encode(
    y=alt.Y(
        'label:N',
        axis=alt.Axis(domain=False, ticks=False, orient='right'), title=None
    ),
    color=alt.Color('color', legend=None)
)

chart_2_legendDomain = ['Tageswert', '7d Moving Average']
chart_2_legendColors = ['#1f77b4', 'lightblue']
chart_2_legendData = pd.DataFrame({
    'label': chart_2_legendDomain
})

legend2 = alt.Chart(chart_2_legendData).mark_square(size=150).encode(
    y=alt.Y(
        'label:N',
        axis=alt.Axis(domain=False, ticks=False, orient='right'),
        title=None
    ),
    color=alt.Color(
        'label',
        scale=alt.Scale(domain=chart_2_legendDomain,range=chart_2_legendColors),
        legend=None
    )
)

chart_2 = (
    new_cases_by_day_chart |
    new_cases_by_day_percent_chart |
    new_cases_by_day_percent_chart2 |
    legend2
).properties(
    padding=0
).configure_axis(
    grid=True,
    gridColor="#eee",
    domainColor="#ddd",
    tickColor="#ddd",
    labelColor="gray",
    labelBound=True,
    minExtent=50
).configure_view(
    strokeWidth=0
).configure_title(
    fontSize=12,
    fontWeight='normal',
    anchor='start',
    color='gray',
    dx=0,
    dy=0
)
chart_2

alt.HConcatChart(...)

In [26]:
new_cases_by_day_doubling_rate_all_cases = chart_with_moving_average(
    valueCalc='log(2)/log(1 + ((max(1,datum.rolling_max - datum.rolling_min)) / (datum.rolling_max)))',
    title='Verdopplungsrate aller bestätigten Fälle',
    yAxisFormat='2s',
    yTextFormat='.2s',
    tooltipLabel='Verdopplungsrate'
)

new_cases_by_day_doubling_rate_active_cases = chart_with_moving_average(
    valueCalc='log(2)/log(1 + ((max(1,datum.rolling_max - datum.rolling_min)) / (datum.rolling_active)))',
    title='Verdopplungsrate aktueller Fälle',
    yAxisFormat='2s',
    yTextFormat='.2s',
    tooltipLabel='Verdopplungsrate'
)

chart_2_doubling_rate = (
    new_cases_by_day_doubling_rate_all_cases | 
    new_cases_by_day_doubling_rate_active_cases |
    legend2
).configure_axis(
    grid=True,
    gridColor="#eee",
    domainColor="#ddd",
    tickColor="#ddd",
    labelColor="gray",
    labelBound=True,
    minExtent=50
).configure_view(
    strokeWidth=0
).configure_title(
    fontSize=12,
    fontWeight='normal',
    anchor='start',
    color='gray',
    dx=50,
    dy=0
)
chart_2_doubling_rate

alt.HConcatChart(...)

In [25]:
def chart_with_moving_average_test(valueCalc, title, yAxisFormat, yTextFormat, tooltipLabel):
    c2a_base = alt.Chart(df, width=150, height=150
    ).transform_timeunit(
        date='yearmonthdate(dataTime)'
    ).transform_aggregate(
        max_total_tests='max(total_tests)',
        max_total_confirmed='max(total_confirmed)',
        groupby=['date']
    ).transform_window(
        rolling_max='max(max_total_tests)',
        rolling_min='min(max_total_tests)',
        rolling_max_confirmed='max(max_total_confirmed)',
        rolling_min_confirmed='min(max_total_confirmed)',
        frame=[-1, 0]
    ).transform_calculate(
        diff=valueCalc
    )

    day = c2a_base.mark_line(
        color='#f46ec6',
        size=2,
        strokeJoin='round'
    ).encode(
        alt.X('date:T', title=''),
        alt.Y('diff:Q', title='', impute=alt.ImputeParams(method='mean')),
        tooltip=['date:T', 'diff:Q']
    )

    c2a_the_tooltip = c2a_base.mark_circle(
        color='#f46ec6',
        size=200,
        opacity=0
    ).encode(
        alt.X('date:T', title=''),
        alt.Y('diff:Q', title=''),
        tooltip=[alt.Tooltip('date:T', title='Datum'), alt.Tooltip(
            'diff:Q', title=tooltipLabel, format=yTextFormat
        )]
    )

    c2a_the_highlight = c2a_base.transform_window(
        sort=[alt.SortField("date", order="descending")], 
        rank="rank(date)"
    ).transform_filter(
        alt.datum.rank == 1
    )

    c2a_the_text = c2a_the_highlight.mark_text(
        align='left',
        color='#f46ec6',
        dx=5,dy=-5,
    ).encode(
        alt.X('date:T'),
        alt.Y('diff:Q', title=''),
        text=alt.Text("diff:Q", format=yTextFormat)
    )

    c2a_the_mark = c2a_the_highlight.mark_circle(
        color='#f46ec6',
        opacity=1
    ).encode(
        alt.X('date:T'),
        alt.Y('diff:Q', title=''),
    )

    ma_base = c2a_base.transform_window(
        rolling_mean='mean(diff)',
        frame=[-7, 0]
    )

    ma = ma_base.mark_line(
        color='#f4b1dd',
        size=2,
        strokeJoin='round'
    ).encode(
        alt.X('date:T', title=''),
        alt.Y('rolling_mean:Q', axis=alt.Axis(format=yAxisFormat, title='')),
    )

    c2b_the_tooltip = ma_base.mark_circle(
        color='#f4b1dd',
        size=200,
        opacity=0
    ).encode(
        alt.X('date:T', title=''),
        alt.Y('rolling_mean:Q', title=''),
        tooltip=[alt.Tooltip('date:T', title='Datum'), alt.Tooltip(
            'rolling_mean:Q', title='7d Moving Average', format=yTextFormat
        )]
    )

    c2b_the_highlight = ma_base.transform_window(
        sort=[alt.SortField("date", order="descending")], 
        rank="rank(date)"
    ).transform_filter(
        alt.datum.rank == 1
    )

    c2b_the_text = c2b_the_highlight.mark_text(
        align='left',
        color='#f4b1dd',
        dx=5,dy=5,
    ).encode(
        alt.X('date:T'),
        alt.Y('rolling_mean:Q', title=''),
        text=alt.Text("rolling_mean:Q", format=yTextFormat)
    )

    c2b_the_mark = c2b_the_highlight.mark_circle(
        color='#f4b1dd',
        opacity=1
    ).encode(
        alt.X('date:T'),
        alt.Y('rolling_mean:Q', title='')
    )

    new_cases_by_day = (
        ma + day + c2a_the_text + c2a_the_mark + c2b_the_text + c2b_the_mark + c2b_the_tooltip + c2a_the_tooltip
    ).properties(title=title)
    
    return new_cases_by_day

new_tests_by_day_chart = chart_with_moving_average_test(
    valueCalc='(datum.rolling_max - datum.rolling_min)',
    title='Neue Tests pro Tag',
    yAxisFormat='2f',
    yTextFormat='.2f',
    tooltipLabel='Neue Tests',
)
new_tests_by_day_percent_chart = chart_with_moving_average_test(
    valueCalc='(datum.rolling_max - datum.rolling_min) / datum.rolling_max',
    title='Steigerung Tests',
    yAxisFormat='2%',
    yTextFormat='.2%',
    tooltipLabel='Steigerung'
)

new_tests_by_day_percent_chart2 = chart_with_moving_average_test(
    valueCalc='(datum.rolling_max_confirmed - datum.rolling_min_confirmed) / (datum.rolling_max - datum.rolling_min)',
    title='Verhältnis Neue Tests / Neue Fälle',
    yAxisFormat='2%',
    yTextFormat='.2%',
    tooltipLabel='Verhältnis'
)

legendData = pd.DataFrame({
    'label': ['Tageswert', '7d Moving Average'],
    'color': ['#f46ec6', '#f4b1dd']
})

legend = alt.Chart(legendData).mark_square(size=150).encode(
    y=alt.Y(
        'label:N',
        axis=alt.Axis(domain=False, ticks=False, orient='right'), title=None
    ),
    color=alt.Color('color', legend=None)
)

chart_3test_legendDomain = ['Tageswert', '7d Moving Average']
chart_3test_legendColors = ['#f46ec6', '#f4b1dd']
chart_3test_legendData = pd.DataFrame({
    'label': chart_3test_legendDomain
})

legend3test = alt.Chart(chart_3test_legendData).mark_square(size=150).encode(
    y=alt.Y(
        'label:N',
        axis=alt.Axis(domain=False, ticks=False, orient='right'),
        title=None
    ),
    color=alt.Color(
        'label',
        scale=alt.Scale(domain=chart_3test_legendDomain,range=chart_3test_legendColors),
        legend=None
    )
)

chart_2test = (
    new_tests_by_day_chart |
    new_tests_by_day_percent_chart |
    new_tests_by_day_percent_chart2 |
    legend3test).configure_axis(
    grid=True,
    gridColor="#eee",
    domainColor="#ddd",
    tickColor="#ddd",
    labelColor="gray",
    labelBound=True,
    minExtent=50
).configure_view(
    strokeWidth=0
).configure_title(
    fontSize=12,
    fontWeight='normal',
    anchor='start',
    color='gray',
    dx=50,
    dy=0
)
chart_2test

alt.HConcatChart(...)

In [8]:
death_day_base = alt.Chart(df, width=300, height=200
).transform_timeunit(
    date='yearmonthdate(dataTime)'
).transform_aggregate(
    max_total_death='max(total_death)',
    groupby=['date']
).transform_window(
    rolling_max='max(max_total_death)',
    rolling_min='min(max_total_death)',
    frame=[-1, 0]
).transform_calculate(
    diff='datum.rolling_max - datum.rolling_min'
)

death_day_line = death_day_base.mark_line(
    color='#d62728',
    size=2,
    strokeJoin='round'
).encode(
    alt.X('date:T', title=''),
    alt.Y('diff:Q', title=''),
    tooltip=['date:T', 'diff:Q']
)

death_day_the_tooltip = death_day_base.mark_circle(
    color='#d62728',
    size=200,
    opacity=0
).encode(
    alt.X('date:T', title=''),
    alt.Y('diff:Q', title=''),
    tooltip=[alt.Tooltip('date:T', title='Datum'), alt.Tooltip('diff:Q', title='Neue Todesfälle')]
)

death_day_the_highlight = death_day_base.transform_window(
    sort=[alt.SortField("date", order="descending")], 
    rank="rank(date)"
).transform_filter(
    alt.datum.rank == 1
)

death_day_the_text = death_day_the_highlight.mark_text(
    align='left',
    color='#d62728',
    dx=5
).encode(
    alt.X('date:T'),
    alt.Y('diff:Q', title=''),
    text=alt.Text("diff:Q")
)

death_day_the_mark = death_day_the_highlight.mark_circle(
    color='#d62728',
    opacity=1
).encode(
    alt.X('date:T'),
    alt.Y('diff:Q', title=''),
)

death_ma_base = death_day_base.transform_window(
    rolling_mean='mean(diff)',
    frame=[-7, 0]
)

death_ma_line = death_ma_base.mark_line(
    color='#ff9896',
    size=2,
    strokeJoin='round'
).encode(
    alt.X('date:T', title=''),
    alt.Y('rolling_mean:Q', title=''),
)

death_ma_the_tooltip = death_ma_base.mark_circle(
    color='#ff9896',
    size=200,
    opacity=0
).encode(
    alt.X('date:T', title=''),
    alt.Y('rolling_mean:Q', title=''),
    tooltip=[alt.Tooltip('date:T', title='Datum'), alt.Tooltip('rolling_mean:Q', title='7d Moving Average')]
)

death_ma_the_highlight = death_ma_base.transform_window(
    sort=[alt.SortField("date", order="descending")], 
    rank="rank(date)"
).transform_filter(
    alt.datum.rank == 1
)

death_ma_the_text = death_ma_the_highlight.mark_text(
    align='left',
    color='#ff9896',
    dx=5
).encode(
    alt.X('date:T'),
    alt.Y('rolling_mean:Q', title=''),
    text=alt.Text("rolling_mean:Q", format=",.2f")
)

death_ma_the_mark = death_ma_the_highlight.mark_circle(
    color='#ff9896',
    opacity=1
).encode(
    alt.X('date:T'),
    alt.Y('rolling_mean:Q', title='')
)

new_death_by_day = (
    death_ma_line + death_day_line + death_day_the_text +
    death_day_the_mark + death_ma_the_text + death_ma_the_mark +
    death_ma_the_tooltip + death_day_the_tooltip
).properties(title='Neue Todesfälle pro Tag')

death_percent_base = alt.Chart(df, width=300, height=200
).transform_timeunit(
    date='yearmonthdate(dataTime)'
).transform_aggregate(
    max_total_death='max(total_death)',
    groupby=['date']
).transform_window(
    rolling_max='max(max_total_death)',
    rolling_min='min(max_total_death)',
    frame=[-1, 0]
).transform_calculate(
    diff='(datum.rolling_max - datum.rolling_min) / datum.rolling_max'
)

death_percent_line = death_percent_base.mark_line(
    color='#d62728',
    size=2,
    strokeJoin='round'
).encode(
    alt.X('date:T', title=''),
    alt.Y('diff:Q', axis=alt.Axis(format='%', title=''), scale=alt.Scale(domain=[0, 1]))
)

death_percent_the_tooltip = death_percent_base.mark_circle(
    color='#d62728',
    size=200,
    opacity=0
).encode(
    alt.X('date:T', title=''),
    alt.Y('diff:Q', title=''),
    tooltip=[alt.Tooltip('date:T', title='Datum'), alt.Tooltip('diff:Q', title='Steigerung %', format=".2%")]
)

death_percent_the_highlight = death_percent_base.transform_window(
    sort=[alt.SortField("date", order="descending")], 
    rank="rank(date)"
).transform_filter(
    alt.datum.rank == 1
)

death_percent_the_text = death_percent_the_highlight.mark_text(
    align='left',
    color='#d62728',
    dx=5,dy=5
).encode(
    alt.X('date:T'),
    alt.Y('diff:Q', title=''),
    text=alt.Text("diff:Q", format=".2%")
)

death_percent_the_mark = death_percent_the_highlight.mark_circle(
    color='#d62728',
    opacity=1
).encode(
    alt.X('date:T'),
    alt.Y('diff:Q', title=''),
)

death_ma_percent = death_percent_base.transform_window(
    rolling_mean='mean(diff)',
    frame=[-7, 0]
).mark_line(
    color='#ff9896',
    size=2,
    strokeJoin='round'
).encode(
    alt.X('date:T', title=''),
    alt.Y('rolling_mean:Q', axis=alt.Axis(format='%', title='')),
)

death_ma_percent_the_tooltip = death_ma_percent.mark_circle(
    color='#ff9896',
    size=200,
    opacity=0
).encode(
    alt.X('date:T', title=''),
    alt.Y('rolling_mean:Q', title=''),
    tooltip=[alt.Tooltip('date:T', title='Datum'), alt.Tooltip('rolling_mean:Q', title='7d Moving Average', format=".2%")]
)


death_ma_percent_the_highlight = death_ma_percent.transform_window(
    sort=[alt.SortField("date", order="descending")], 
    rank="rank(date)"
).transform_filter(
    alt.datum.rank == 1
)

death_ma_percent_the_text = death_ma_percent_the_highlight.mark_text(
    align='left',
    color='#ff9896',
    dx=5,dy=-5
).encode(
    alt.X('date:T'),
    alt.Y('rolling_mean:Q', title=''),
    text=alt.Text("rolling_mean:Q", format=".2%")
)

death_ma_percent_the_mark = death_ma_percent_the_highlight.mark_circle(
    color='#ff9896',
    opacity=1
).encode(
    alt.X('date:T'),
    alt.Y('rolling_mean:Q', title='')
)

new_death_percent_by_day = (
    death_ma_percent + death_percent_line +
    death_ma_percent_the_text + death_ma_percent_the_mark +
    death_percent_the_text + death_percent_the_mark +
    death_ma_percent_the_tooltip + death_percent_the_tooltip
).properties(title='Neue Todesfälle akkumuliert Steigerung % zum Vortag')

legendData_3 = pd.DataFrame({
    'label': ['Tageswert', '7d Moving Average'],
    'color': ['#d62728', '#ff9896']
})

legend_3 = alt.Chart(legendData_3).mark_square(size=150).encode(
    y=alt.Y(
        'label:N',
        axis=alt.Axis(domain=False, ticks=False, orient='right'), title=None
    ),
    color=alt.Color('color', legend=None)
)

chart_3_legendDomain = ['Tageswert', '7d Moving Average']
chart_3_legendColors = ['#d62728', '#ff9896']
chart_3_legendData = pd.DataFrame({
    'label': chart_3_legendDomain
})

legend3 = alt.Chart(chart_3_legendData).mark_square(size=150).encode(
    y=alt.Y(
        'label:N',
        axis=alt.Axis(domain=False, ticks=False, orient='right'),
        title=None
    ),
    color=alt.Color(
        'label',
        scale=alt.Scale(domain=chart_3_legendDomain,range=chart_3_legendColors),
        legend=None
    )
)

chart_3 = (new_death_by_day | new_death_percent_by_day | legend3).configure_axis(
    grid=True,
    gridColor="#eee",
    domainColor="#ddd",
    tickColor="#ddd",
    labelColor="gray",
    labelBound=True,
    minExtent=50
).configure_view(
    strokeWidth=0
).configure_title(
    fontSize=12,
    fontWeight='normal',
    anchor='start',
    color='gray'
)
chart_3

alt.HConcatChart(...)

In [19]:
rec_day_base = alt.Chart(df, width=300, height=200
).transform_timeunit(
    date='yearmonthdate(dataTime)'
).transform_aggregate(
    max_total_recovered='max(total_recovered)',
    groupby=['date']
).transform_window(
    rolling_max='max(max_total_recovered)',
    rolling_min='min(max_total_recovered)',
    frame=[-1, 0]
).transform_calculate(
    diff='datum.rolling_max - datum.rolling_min'
)

rec_day_line = rec_day_base.mark_line(
    color='#ff7f0e',
    size=2,
    strokeJoin='round'
).encode(
    alt.X('date:T', title=''),
    alt.Y('diff:Q', title=''),
    tooltip=['date:T', 'diff:Q']
)

rec_day_the_tooltip = rec_day_base.mark_circle(
    color='#ff7f0e',
    size=200,
    opacity=0
).encode(
    alt.X('date:T', title=''),
    alt.Y('diff:Q', title=''),
    tooltip=[alt.Tooltip('date:T', title='Datum'), alt.Tooltip('diff:Q', title='Neue Genesene')]
)

rec_day_the_highlight = rec_day_base.transform_window(
    sort=[alt.SortField("date", order="descending")], 
    rank="rank(date)"
).transform_filter(
    alt.datum.rank == 1
)

rec_day_the_text = rec_day_the_highlight.mark_text(
    align='left',
    color='#ff7f0e',
    dx=5
).encode(
    alt.X('date:T'),
    alt.Y('diff:Q', title=''),
    text=alt.Text("diff:Q")
)

rec_day_the_mark = rec_day_the_highlight.mark_circle(
    color='#ff7f0e',
    opacity=1
).encode(
    alt.X('date:T'),
    alt.Y('diff:Q', title=''),
)

rec_ma_base = rec_day_base.transform_window(
    rolling_mean='mean(diff)',
    frame=[-7, 0]
)

rec_ma_line = rec_ma_base.mark_line(
    color='#ffbb78',
    size=2,
    strokeJoin='round'
).encode(
    alt.X('date:T', title=''),
    alt.Y('rolling_mean:Q', title=''),
)

rec_ma_the_tooltip = rec_ma_base.mark_circle(
    color='#ffbb78',
    size=200,
    opacity=0
).encode(
    alt.X('date:T', title=''),
    alt.Y('rolling_mean:Q', title=''),
    tooltip=[alt.Tooltip('date:T', title='Datum'), alt.Tooltip('rolling_mean:Q', title='7d Moving Average')]
)

rec_ma_the_highlight = rec_ma_base.transform_window(
    sort=[alt.SortField("date", order="descending")], 
    rank="rank(date)"
).transform_filter(
    alt.datum.rank == 1
)

rec_ma_the_text = rec_ma_the_highlight.mark_text(
    align='left',
    color='#ffbb78',
    dx=5
).encode(
    alt.X('date:T'),
    alt.Y('rolling_mean:Q', title=''),
    text=alt.Text("rolling_mean:Q", format=",.2f")
)

rec_ma_the_mark = rec_ma_the_highlight.mark_circle(
    color='#ffbb78',
    opacity=1
).encode(
    alt.X('date:T'),
    alt.Y('rolling_mean:Q', title='')
)

new_rec_by_day = (
    rec_ma_line + rec_day_line + rec_day_the_text +
    rec_day_the_mark + rec_ma_the_text + rec_ma_the_mark +
    rec_ma_the_tooltip + rec_day_the_tooltip
).properties(title='Neue Genesene pro Tag')

rec_percent_base = alt.Chart(df, width=300, height=200
).transform_timeunit(
    date='yearmonthdate(dataTime)'
).transform_aggregate(
    max_total_recovered='max(total_recovered)',
    groupby=['date']
).transform_window(
    rolling_max='max(max_total_recovered)',
    rolling_min='min(max_total_recovered)',
    frame=[-1, 0]
).transform_calculate(
    diff='(datum.rolling_max - datum.rolling_min) / datum.rolling_max'
)

rec_percent_line = rec_percent_base.mark_line(
    color='#ff7f0e',
    size=2,
    strokeJoin='round'
).encode(
    alt.X('date:T', title=''),
    alt.Y('diff:Q', axis=alt.Axis(format='%', title=''), scale=alt.Scale(domain=[0, 1]))
)

rec_percent_the_tooltip = rec_percent_base.mark_circle(
    color='#ff7f0e',
    size=200,
    opacity=0
).encode(
    alt.X('date:T', title=''),
    alt.Y('diff:Q', title=''),
    tooltip=[alt.Tooltip('date:T', title='Datum'), alt.Tooltip('diff:Q', title='Steigerung %', format=".2%")]
)

rec_percent_the_highlight = rec_percent_base.transform_window(
    sort=[alt.SortField("date", order="descending")], 
    rank="rank(date)"
).transform_filter(
    alt.datum.rank == 1
)

rec_percent_the_text = rec_percent_the_highlight.mark_text(
    align='left',
    color='#ff7f0e',
    dx=5,dy=5
).encode(
    alt.X('date:T'),
    alt.Y('diff:Q', title=''),
    text=alt.Text("diff:Q", format=".2%")
)

rec_percent_the_mark = rec_percent_the_highlight.mark_circle(
    color='#ff7f0e',
    opacity=1
).encode(
    alt.X('date:T'),
    alt.Y('diff:Q', title=''),
)

rec_ma_percent = rec_percent_base.transform_window(
    rolling_mean='mean(diff)',
    frame=[-7, 0]
).mark_line(
    color='#ffbb78',
    size=2,
    strokeJoin='round'
).encode(
    alt.X('date:T', title=''),
    alt.Y('rolling_mean:Q', axis=alt.Axis(format='%', title='')),
)

rec_ma_percent_the_tooltip = rec_ma_percent.mark_circle(
    color='#ffbb78',
    size=200,
    opacity=0
).encode(
    alt.X('date:T', title=''),
    alt.Y('rolling_mean:Q', title=''),
    tooltip=[alt.Tooltip('date:T', title='Datum'), alt.Tooltip('rolling_mean:Q', title='7d Moving Average', format=".2%")]
)


rec_ma_percent_the_highlight = rec_ma_percent.transform_window(
    sort=[alt.SortField("date", order="descending")], 
    rank="rank(date)"
).transform_filter(
    alt.datum.rank == 1
)

rec_ma_percent_the_text = rec_ma_percent_the_highlight.mark_text(
    align='left',
    color='#ffbb78',
    dx=5,dy=-5
).encode(
    alt.X('date:T'),
    alt.Y('rolling_mean:Q', title=''),
    text=alt.Text("rolling_mean:Q", format=".2%")
)

rec_ma_percent_the_mark = rec_ma_percent_the_highlight.mark_circle(
    color='#ffbb78',
    opacity=1
).encode(
    alt.X('date:T'),
    alt.Y('rolling_mean:Q', title='')
)

new_rec_percent_by_day = (
    rec_ma_percent + rec_percent_line +
    rec_ma_percent_the_text + rec_ma_percent_the_mark +
    rec_percent_the_text + rec_percent_the_mark +
    rec_ma_percent_the_tooltip + rec_percent_the_tooltip
).properties(title='Neue Genesene akkumuliert Steigerung % zum Vortag')

chart_4_legendDomain = ['Tageswert', '7d Moving Average']
chart_4_legendColors = ['#ff7f0e', '#ffbb78']
chart_4_legendData = pd.DataFrame({
    'label': chart_4_legendDomain
})

legend4 = alt.Chart(chart_4_legendData).mark_square(size=150).encode(
    y=alt.Y(
        'label:N',
        axis=alt.Axis(domain=False, ticks=False, orient='right'),
        title=None
    ),
    color=alt.Color(
        'label',
        scale=alt.Scale(domain=chart_4_legendDomain,range=chart_4_legendColors),
        legend=None
    )
)

chart_4 = (new_rec_by_day | new_rec_percent_by_day | legend4).configure_axis(
    grid=True,
    gridColor="#eee",
    domainColor="#ddd",
    tickColor="#ddd",
    labelColor="gray",
    labelBound=True,
    minExtent=50
).configure_view(
    strokeWidth=0
).configure_title(
    fontSize=12,
    fontWeight='normal',
    anchor='start',
    color='gray'
)
chart_4

alt.HConcatChart(...)

In [10]:
# save chart_1 as formatted JSON
with open('../data/chart_1.json', 'w') as jsonFile:
    json.dump(json.loads(chart_1.to_json()), jsonFile, indent=4, sort_keys=True)

In [11]:
# save chart_2 as formatted JSON
with open('../data/chart_2.json', 'w') as jsonFile:
    json.dump(json.loads(chart_2.to_json()), jsonFile, indent=4, sort_keys=True)

In [12]:
# save chart_2_doubling_rate as formatted JSON
with open('../data/chart_2_doubling_rate.json', 'w') as jsonFile:
    json.dump(json.loads(chart_2_doubling_rate.to_json()), jsonFile, indent=4, sort_keys=True)

In [13]:
# save chart_2b as formatted JSON
with open('../data/chart_2test.json', 'w') as jsonFile:
    json.dump(json.loads(chart_2test.to_json()), jsonFile, indent=4, sort_keys=True)

In [14]:
# save chart_3 as formatted JSON
with open('../data/chart_3.json', 'w') as jsonFile:
    json.dump(json.loads(chart_3.to_json()), jsonFile, indent=4, sort_keys=True)

In [15]:
# save chart_4 as formatted JSON
with open('../data/chart_4.json', 'w') as jsonFile:
    json.dump(json.loads(chart_4.to_json()), jsonFile, indent=4, sort_keys=True)